In [1]:
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
import random
import sys
import copy
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import HeteroData
from tqdm import tqdm
import json
from itertools import combinations
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


ModuleNotFoundError: ignored

In [ ]:
!pip install torch_geometric

In [ ]:
PATH_FOLDER = "/content/drive/MyDrive/Darmoth-Cre-Gene-Project"
PATH_CRE_ATTRIBUTES = "Dataset/LZ_cRE_attributes/LZ_chr1.txt"
PATH_DNA = "Dataset/cRE_DNA/chr1.npy"
PATH_EDGE = "Dataset/LZ_cREedge/LZ_t4_chr1.txt"

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path = PATH_FOLDER
os.chdir(path)

In [ ]:
import os

In [ ]:
#import dataset
cREatt= pd.read_table(PATH_CRE_ATTRIBUTES,header=None,sep=" ")

cREatt1=cREatt.iloc[:,[3,5,6,7,8,9,10,11,12,13,14]]
cREatt1.to_numpy()

cRE_att = torch.tensor(StandardScaler().fit_transform(cREatt1.to_numpy()), dtype=torch.float)

cRE_DNA=np.load(PATH_DNA)

cRE_edge = pd.read_table(PATH_EDGE,header=None,sep=" ")

cRE_edgeindex=torch.tensor([cRE_edge[:][0],cRE_edge[:][1]], dtype=torch.long)

cREIsEdge=2*torch.eye(cRE_att.size()[0],cRE_att.size()[0])-1

for i in range(int(len(cRE_edgeindex[1]))):
    cREIsEdge[cRE_edgeindex[0][i]][cRE_edgeindex[1][i]]=1
    cREIsEdge[cRE_edgeindex[1][i]][cRE_edgeindex[0][i]]=1



In [ ]:
#generate positive/negative label matrix


portion=0.1
cRE_inputIsEdge=cREIsEdge
cRE_hiddenInput=random.sample(range(len(cRE_edgeindex[1])),k=int(len(cRE_edge[1])*portion))

for i in cRE_hiddenInput:
    cREIsEdge[cRE_edgeindex[0][i]][cRE_edgeindex[1][i]] =-1
    cRE_inputIsEdge[cRE_edgeindex[1][i]][cRE_edgeindex[0][i]]=-1
num_neg=0
m=cRE_att.size()[0]-1
avil=torch.eye(cRE_att.size()[0],cRE_att.size()[0])
maxneg=int(len(cRE_edgeindex[1])) * 5
while(num_neg < maxneg):
    t=random.randint(0,m*m)
    a=t//m
    b=t%m
    if(cREIsEdge[a][b]==-1 and avil[a][b]==0):
        cREIsEdge[a][b]=0
        cREIsEdge[b][a]=0
        cRE_inputIsEdge[a][b]=0
        cRE_inputIsEdge[b][a]=0
        num_neg=num_neg+1
        avil[a][b]=1
        avil[b][a]=1
        

In [ ]:
#generate dataset (half node are close to each other, the other half are sporadic)

radius=250      
listlength=1000
randomsize=50
neighborsize=50
samplesize=randomsize+neighborsize
dataset=[]
for i in tqdm(range(listlength)):
    x_dataset=[]
    x1_dataset=[]
    edge_index=[]
    edgelista=[]
    edgelistb=[]
    y=[-1]*(samplesize**2)
    xi=[]
    center=random.choice(range(cRE_att.shape[0]))
    center=radius if center<radius else center
    center=cRE_att.shape[0]-radius if center+radius>=cRE_att.shape[0] else center
#     xi=random.sample(range(cRE_att.shape[0]),k=samplesize)
#     xi1=random.sample((range(samplesize)),k=samplesize//2)
#     xi2=list(set((range(samplesize)))-set(xi1))
    #xi2=[var for var in list(range(samplesize))]
    xi1=random.sample(range(center-radius,center+radius),k=neighborsize)
    xi2=(random.sample(list(set(range(cRE_att.shape[0]))-set(xi1)),k=neighborsize))
    xi=xi1+xi2
    xi3=random.sample((range(samplesize)),k=samplesize//2)
    xi4=list(set((range(samplesize)))-set(xi3))    
    XI=xi3+xi4
    for m in xi3:
        for n in xi4:
            if(cREIsEdge[xi[XI[m]]][xi[XI[n]]]==1):
                edgelista.append(m)
                edgelistb.append(n)
                edgelista.append(n)
                edgelistb.append(m)
            y[m*(samplesize)+n]=cREIsEdge[xi[XI[m]]][xi[XI[n]]]
            y[n*(samplesize)+m]=cREIsEdge[xi[XI[m]]][xi[XI[n]]]
    for j in range(samplesize):
        x_dataset.append(cRE_att[xi[XI[j]]].tolist()+cRE_DNA[xi[XI[j]]].tolist())
    edge_index = torch.tensor([edgelista,edgelistb], dtype=torch.long)
    dataset.append(Data(x=torch.tensor(np.array(x_dataset)), edge_index=torch.tensor(np.array(edge_index)).long(),y=torch.tensor(np.array(y)).long()))


from sklearn.model_selection import train_test_split

dataset_train,dataset_test = train_test_split(dataset, test_size=0.20, random_state=42)

100%|██████████| 1000/1000 [02:54<00:00,  5.73it/s]


In [ ]:
#model training and eval
class network(torch.nn.Module):
    def __init__(self,indexsize,gcnInputsize,gcnHiddensize,gcnOutputsize,batchSize):
        super(network, self).__init__()
        self.conv1 = GCNConv(gcnInputsize, gcnHiddensize)
        self.conv2 = GCNConv(gcnHiddensize, gcnOutputsize)
        self.matrix=Variable(torch.randn(gcnOutputsize,gcnOutputsize).type(torch.FloatTensor),requires_grad=True)
        self.linear=torch.nn.Linear(1,2)
        self.transform1=nn.Linear(indexsize, gcnInputsize)
        self.transform2=nn.Linear(768,gcnInputsize)
        self.indexsize=indexsize       
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x1=x[:,:self.indexsize].float()
        x2=x[:,self.indexsize:].float()
        x = self.transform1(x1)+self.transform2(x2)
        x = self.conv1(x.float(), edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        ypred=[]
        for i in range(x.size(0)//samplesize):
            x1=x[i*samplesize:(i+1)*samplesize,:]
            ypred.append(self.linear(torch.matmul(torch.matmul(x1,self.matrix),x1.t()).unsqueeze(-1)).view(-1,2))
        
        return torch.stack(ypred).view(-1,2)
    
batchSize=16
feature=torch.tensor([10,9,8,7,6,5,4,3,2,1,0])

key_list=['A','B','C','D','E','F','G','H','I','J','K']
# key_list=['K','J','I','H','G','F','E','D','C','B','A']
dict_result={}
gcnInputsize=500
gcnHiddensize=300
gcnOutputsize=200
for i in range(11,12):
    featureIndex=torch.tensor(list(combinations(feature,r=i)))
    for index in tqdm(featureIndex):
        key="".join([key_list[i] for i in index])
        # print(key)
        # print(index)
        model=network(index.size(0),gcnInputsize,gcnHiddensize,gcnOutputsize,batchSize)
#         criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([4,1]).float(),ignore_index=-1)
        criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

        index1=torch.cat([index,torch.Tensor(list(range(11,779))).long()],dim=0)
        dataset_train1 = [Data(x = torch.index_select(d.x, 1, index1), edge_index=d.edge_index, y=d.y) for d in dataset_train]
        loader = DataLoader(dataset_train1, batch_size=batchSize, shuffle=True)

        for epoch in range(200):
            count=0
            loss=0
            for data1 in loader:
                print('data1 shape is ',data1)
                pred_y = model(data1)
                print('pred_y shaoe',pred_y.shape)
                print('data1.shape',len(data1.y))
                loss = loss + criterion(pred_y, data1.y)
                count=count+1
                if(count==10):
                    loss.backward()
                    optimizer.step()
                    print('epoch {}, loss {}'.format(epoch, loss.item()))
                    optimizer.zero_grad()
                    loss=0
                    count=0 
            if(loss!=0):
                loss.backward()
                optimizer.step()
                print('epoch {}, loss {}'.format(epoch, loss.item()))
                optimizer.zero_grad()
        index1=torch.cat([index,torch.Tensor(list(range(11,779))).long()],dim=0)                
        dataset_test1 = [Data(x = torch.index_select(d.x, 1, index1), edge_index=d.edge_index, y=d.y) for d in dataset_test]
        testloader = DataLoader(dataset_test1, batch_size=batchSize, shuffle=True)
        from sklearn.metrics import f1_score
        ytrue=[]
        ypred=[]
        with torch.no_grad():    
            for data in testloader:
                pred_y=model(data)
                v,i=torch.max(pred_y,dim=1)
                ytrue+=data.y.tolist()
                ypred+=i
        ypred1 = [ypred[i] for i in range(len(ypred)) if ytrue[i] != -1] 
        ytrue1 = [var for var in ytrue if var != -1] 
        f1=f1_score(ytrue1,ypred1,average='macro')
        print(f1)      
        dict_result.update({key:f1})
        out_file = open("dict_result_"+key+".json", "w")
          
        json.dump(dict_result, out_file, indent = 6)
          
        out_file.close()
 

  0%|          | 0/1 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
data1 shape is  DataBatch(x=[1600, 779], edge_index=[2, 476], y=[160000], batch=[1600], ptr=[17])
pred_y shaoe torch.Size([160000, 2])
data1.shape 160000
data1 shape is  DataBatch(x=[1600, 779], edge_index=[2, 450], y=[160000], batch=[1600], ptr=[17])
pred_y shaoe torch.Size([160000, 2])
data1.shape 160000
data1 shape is  DataBatch(x=[1600, 779], edge_index=[2, 448], y=[160000], batch=[1600], ptr=[17])
pred_y shaoe torch.Size([160000, 2])
data1.shape 160000
epoch 111, loss 1.4260421991348267
data1 shape is  DataBatch(x=[1600, 779], edge_index=[2, 456], y=[160000], batch=[1600], ptr=[17])
pred_y shaoe torch.Size([160000, 2])
data1.shape 160000
data1 shape is  DataBatch(x=[1600, 779], edge_index=[2, 524], y=[160000], batch=[1600], ptr=[17])
pred_y shaoe torch.Size([160000, 2])
data1.shape 160000
data1 shape is  DataBatch(x=[1600, 779], edge_index=[2, 462], y=[160000], batch=[1600], ptr=[17])
pred_y shaoe torch.Size([160000, 2])
data1.sha

  0%|          | 0/1 [21:03<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:

truepos=0
falseneg=0
newpos=0
newneg=0
trueneg=0
falsepos=0
for i in range(len(ypred)):
    if(ytrue[i]==1):
        if(ypred[i]==1):
            truepos+=1
        else:
            falseneg+=1
    elif(ytrue[i]==-1):
        if(ypred[i]==1):
            newpos+=1
        else:
            newneg+=1
    elif(ytrue[i]==0):
        if(ypred[i]==1):
            falsepos+=1
        else:
            trueneg+=1
print(truepos,falseneg,newpos,newneg,falsepos,trueneg)



In [ ]:
#plot the whole graph
import copy
prediction=torch.zeros(10000,10000)
samplesize=200
plotsize=100
plotsize2=200
x_cpy = copy.deepcopy(cRE_att)
x_cpy = torch.cat([x_cpy, torch.tensor(cRE_DNA)], 1)
x_cpy = torch.cat([x_cpy, x_cpy[:10000 - x_cpy.size(0)]])
# print(x_cpy.size())
for i in tqdm(range(cRE_att.shape[0]//100+1)):
#     if i==15:
#         break
    for j in range(i+1):
        edge_index=[]
        edgelista=[]
        edgelistb=[]
        x_dataset=torch.cat([x_cpy[i*plotsize:(i+1)*plotsize] , x_cpy[j*plotsize:(j+1)*plotsize]],0)

        edgelista,edgelistb=(cREIsEdge[i*plotsize:(i+1)*plotsize,j*plotsize:(j+1)*plotsize] == 1).nonzero(as_tuple=True)
        edgelistb+=plotsize
        temp=edgelista
        edgelista=torch.cat([edgelista,edgelistb],0)
        edgelistb=torch.cat([edgelistb,temp],0)
        edge_index = torch.stack([edgelista,edgelistb]).long()
        datasetpred=Data(x=x_dataset, edge_index=torch.tensor(np.array(edge_index)).long())
        pred_y=model(datasetpred)
        pred_y = F.softmax(pred_y)
        v,ind=torch.max(pred_y,dim=1)
        pred_y = pred_y.tolist()
        diff = [var[1] - var[0] for var in pred_y]
        ind=[1 if pred_y[k][1] - pred_y[k][0] >= 0.5 else 0 for k in range(len(ind))]
        pred_edge_index = [var[0] - 1 for var in enumerate(ind,1) if var[1] == 1]
        pred_edge_index = [var for var in pred_edge_index if (var % plotsize2) * plotsize2 + (var // plotsize2) in pred_edge_index]
        # print([(var // 200, var % 200) for var in pred_edge_index])
        pred_edge_index_tmp = [var for var in pred_edge_index if (var // plotsize2 < plotsize and var % plotsize2 >= plotsize) or (var // plotsize2 >= plotsize and var % plotsize2 < plotsize)]
        pred_edge_index_tmp = [var if (var // plotsize2) < (var % plotsize2) else (var % plotsize2) * plotsize2 + (var // plotsize2) for var in pred_edge_index_tmp]
        
        tmp = torch.zeros(plotsize,plotsize)
        for point in pred_edge_index_tmp:
            point_x = point // plotsize2
            point_y = point % plotsize2 - plotsize
#             point_rv = (point_y+100)*200 + point_x
#             if point_rv  in pred_edge_index:
            tmp[point_x, point_y] = 1
#        tmp = torch.reshape(v,(100,100)).detach()
        plt.imsave("temp1.pdf",tmp,cmap='bwr')
        prediction[i*plotsize:(i+1)*plotsize, j*plotsize:(j+1)*plotsize] = tmp

plt.imsave("newpred12_0.5.pdf",prediction,cmap='bwr')